---
layout: post
title: Collectable Lesson Starter
description: Usage of compareTo and toString in Collectable framework
courses: { csa: {week: 15} }
type: ccc
---

## Abstract, Comparable, Implements, Interface

Establish class heirarchy, enforce definitions, and use compareTo.

In [3]:
/* This is parent class...
   Objective would be to push more functionality into parent Class 
   - Enforces consistent definition of child Class

 */
public abstract class Collectable implements Comparable <Collectable> {
	public final String masterType = "Collectable";
	private String type;	// extender should define their data type

	/* Enumerated interface of key types 
	 * an interface named KeyTypes is declared with a single method name(). 
	 * the Collectable class contains an abstract method getKey(), 
	 * which must be implemented by its subclasses. 
	 * must provide a method that returns an object implementing the KeyTypes interface.
	*/ 
	public interface KeyTypes {
		String name();
	}
	protected abstract KeyTypes getKey();  	// this method helps force usage of KeyTypes
	protected abstract KeyTypes getSortKey();  	// this method helps force usage of KeyTypes

	// getter
	public String getMasterType() {
		return masterType;
	}

	// getter
	public String getType() {
		return type;
	}

	// setter
	public void setType(String type) {
		this.type = type;
	}
	
	/* This method is used to establish key order
	 * an abstract toString method is declared
	 * this method must be implemented by any class that extends Collectable
	 * toString is used to establish the string representation of an object
	*/  
    public abstract String toJson();
	public abstract String getSortKeyValue();

	/* 'Collectable' requires toString override
	 * toString provides data based off of Static Key setting
	 */
	@Override
	public String toString()
	{
		return toJson();
	}

	/* This method is used to compare toString of objects
	 * the compareTo method is implemented from the Comparable interface
	 * it compares the string representations of two Collectable objects 
	 * using their toString methods
	*/
	public int compareTo(Collectable obj) {
		return this.toString().compareTo(obj.toString());
	}

	// static print method for Collectable Array
	public static void print(Collectable[] objs) {
		// print "Collectable: Objects'
		for(Object o : objs)	// observe that type is Opaque
			System.out.println(o);

		System.out.println();
	}
}

In [4]:
/*
 * Pet class extends Collectable and defines abstract methods
 */
public class Pet extends Collectable {
	// Class data
	public enum KeyType implements KeyTypes {title, name, age, color, catalog}
	public static KeyTypes key = KeyType.title;  // static initializer
	public static KeyTypes sortKey = KeyType.title;
	public static void setOrder(KeyTypes key) { Pet.key = key; }
	public static void sort(Collectable[] c, KeyType key) { 
		Pet.key = key; 
		Pet.sortKey = key; 
		Arrays.sort(c); 
		Pet.key = KeyType.title; 
	}

	// Instance data
	private final String name;
	private final int age;
	private final String color;

	/* constructor
	 *
	 */
	public Pet(String name, int age, String color)
	{
		super.setType("Pet");
		this.name = name;
		this.age = age;
		this.color = color;
	}

	/* 'Collectable' requires getKey to help enforce KeyTypes usage */
	@Override
	protected KeyTypes getKey() { return Pet.key; }
	@Override
	protected KeyTypes getSortKey() { return Pet.sortKey; }

	/* Getters / Accessors
	 * 
	 */
	public String getName() { return this.name; }
	public int getAge() { return this.age; }
	public String getColor() { return this.color; }

	// New method to convert object to JSON format
    public String toJson() {
        // Implement JSON formatting logic here
        // Use the selected sort key to determine the order of fields
        return "{"
				+ "\"" + "key" + "\":\"" + getSortKeyValue() + "\","
				+ "\"" + "data" + "\":{"
                	+ "\"type\":\"" + getType() + "\","
                	+ "\"name\":\"" + getName() + "\","
                	+ "\"age\":" + getAge() + ","
                	+ "\"color\":\"" + getColor()
				+ "}"
                + "}";
    }

	/* 'Collectable' requires getSortKeyValue override
	 * provides key based off of Static Key setting
	 */
	public String getSortKeyValue() {
        if (KeyType.name.equals(getSortKey())) {
            return getName();
        } else if (KeyType.age.equals(getSortKey())) {
			String zeroPad = "00" + getAge();
            return zeroPad.substring(zeroPad.length() - 2);
        } else if (KeyType.color.equals(getSortKey())) {
            return getColor();
		} else if (KeyType.catalog.equals(getSortKey())) {
			return getName() + " " + getColor() + " " + getAge();
        }
        // Default is Type
        return getType();
    }

	// Test data initializer
	public static Pet[] pets() {
		return new Pet[]{
				new Pet("Lion", 8, "Gold"),
				new Pet("Pig", 3, "Pink"),
				new Pet("Robin", 7, "Red"),
				new Pet("Cat", 10, "Black"),
				new Pet("Kitty", 1, "Calico"),
				new Pet("Dog", 14, "Brown")
		};
	}
	
	/* main to test Pet class
	 * 
	 */
	public static void main(String[] args)
	{
		// print with title
		Pet.setOrder(KeyType.title);
		Collectable[] pets = Pet.pets();

		for (Pet.KeyType key : Pet.KeyType.values()) {
			Pet.sort(pets, key);
			Collectable.print(pets);
		}
	}

}
Pet.main(null);

{"key":"Pet","data":{"type":"Pet","name":"Cat","age":10,"color":"Black}}
{"key":"Pet","data":{"type":"Pet","name":"Dog","age":14,"color":"Brown}}
{"key":"Pet","data":{"type":"Pet","name":"Kitty","age":1,"color":"Calico}}
{"key":"Pet","data":{"type":"Pet","name":"Lion","age":8,"color":"Gold}}
{"key":"Pet","data":{"type":"Pet","name":"Pig","age":3,"color":"Pink}}
{"key":"Pet","data":{"type":"Pet","name":"Robin","age":7,"color":"Red}}

{"key":"Cat","data":{"type":"Pet","name":"Cat","age":10,"color":"Black}}
{"key":"Dog","data":{"type":"Pet","name":"Dog","age":14,"color":"Brown}}
{"key":"Kitty","data":{"type":"Pet","name":"Kitty","age":1,"color":"Calico}}
{"key":"Lion","data":{"type":"Pet","name":"Lion","age":8,"color":"Gold}}
{"key":"Pig","data":{"type":"Pet","name":"Pig","age":3,"color":"Pink}}
{"key":"Robin","data":{"type":"Pet","name":"Robin","age":7,"color":"Red}}

{"key":"01","data":{"type":"Pet","name":"Kitty","age":1,"color":"Calico}}
{"key":"03","data":{"type":"Pet","name":"Pig","

In [5]:
/*
 * Car class extends Collectable and defines abstract methods
 */
public class Car extends Collectable {
	// Class data
	public enum KeyType implements KeyTypes {title, make, model, year, color, catalog}
	public static KeyTypes key = KeyType.title;  // static initializer
	public static KeyTypes sortKey = KeyType.title;
	public static void setOrder(KeyTypes key) { Car.key = key; }
	public static void sort(Collectable[] c, KeyType key) { 
		Car.key = key; 
		Car.sortKey = key; 
		Arrays.sort(c); 
		Car.key = KeyType.title; 
	}

	// Instance data
    private final String make;
    private final String model;
	private final int year;
	private final String color;

	/* constructor
	 *
	 */
	public Car(String make, String model, int year, String color)
	{
		super.setType("Car");
		this.make = make;
        this.model = model;
		this.year = year;
		this.color = color;
	}

	/* 'Collectable' requires getKey to help enforce KeyTypes usage */
	@Override
	protected KeyTypes getKey() { return Car.key; }
	@Override
	protected KeyTypes getSortKey() { return Car.sortKey; }

	/* Getters / Accessors
	 * 
	 */
	public String getMake() { return this.make; }
    public String getModel() { return this.model; }
	public int getYear() { return this.year; }
	public String getColor() { return this.color; }

	// Convert object to JSON format
    public String toJson() {
        // Implement JSON formatting logic here
        // Use the selected sort key to determine the order of fields
		return "{"
				+ "\"" + "key" + "\":\"" + getSortKeyValue() + "\","
				+ "\"" + "data" + "\":{"
					+ "\"type\":\"" + getType() + "\","
					+ "\"make\":\"" + getMake() + "\","
					+ "\"model\":\"" + getModel() + "\","
					+ "\"year\":" + getYear() + ","
					+ "\"color\":\"" + getColor()
				+ "}"
				+ "}";
    }

	/* 'Collectable' requires getSortKeyValue override
	 * toString provides data based off of Static Key setting
	 */
	@Override
    public String getSortKeyValue() {
        if (KeyType.make.equals(getSortKey())) {
            return getMake();
        } else if (KeyType.model.equals(getSortKey())) {
            return getModel();
        } else if (KeyType.year.equals(getSortKey())) {
            return String.valueOf(getYear());
        } else if (KeyType.color.equals(getSortKey())) {
            return getColor();
		} else if (KeyType.catalog.equals(getSortKey())) {
			return getMake() + " " + getModel() + " " + getYear();
        }
        // Default is Type
        return getType();
    }

	// Test data initializer
	public static Car[] cars() {
		return new Car[]{
				new Car("Ford", "Fusion", 2015, "Guard"),
				new Car("Ford", "Excursion", 2003, "Green"),
                new Car("Ford", "F-350", 1997, "Green"),
                new Car("Cadillac", "Boughman", 1969, "Black"),
                new Car("Acura", "RL", 2006, "Silver")
		};
	}
	
	/* main to test Car class
	 * 
	 */
	public static void main(String[] args)
	{
		// print with title
		Car.setOrder(KeyType.title);
		Collectable[] cars = Car.cars();

		for (Car.KeyType key : Car.KeyType.values()) {
			Car.sort(cars, key);
			Collectable.print(cars);
		}
	}

}
Car.main(null);

{"key":"Car","data":{"type":"Car","make":"Acura","model":"RL","year":2006,"color":"Silver}}
{"key":"Car","data":{"type":"Car","make":"Cadillac","model":"Boughman","year":1969,"color":"Black}}
{"key":"Car","data":{"type":"Car","make":"Ford","model":"Excursion","year":2003,"color":"Green}}
{"key":"Car","data":{"type":"Car","make":"Ford","model":"F-350","year":1997,"color":"Green}}
{"key":"Car","data":{"type":"Car","make":"Ford","model":"Fusion","year":2015,"color":"Guard}}

{"key":"Acura","data":{"type":"Car","make":"Acura","model":"RL","year":2006,"color":"Silver}}
{"key":"Cadillac","data":{"type":"Car","make":"Cadillac","model":"Boughman","year":1969,"color":"Black}}
{"key":"Ford","data":{"type":"Car","make":"Ford","model":"Excursion","year":2003,"color":"Green}}
{"key":"Ford","data":{"type":"Car","make":"Ford","model":"F-350","year":1997,"color":"Green}}
{"key":"Ford","data":{"type":"Car","make":"Ford","model":"Fusion","year":2015,"color":"Guard}}

{"key":"Boughman","data":{"type":"Ca

In [9]:
import java.util.ArrayList;
import java.util.Collections;
import java.util.List;

public class Collection {
	
    public static void main(String[] args) {
        List<Collectable> collection = new ArrayList<>();

        collection.addAll(List.of(Pet.pets()));
        collection.addAll(List.of(Car.cars()));

        Pet.setOrder(Pet.KeyType.catalog);
        Car.setOrder(Car.KeyType.catalog);

        Collections.sort(collection);

        for (Collectable c : collection) {
            System.out.println(c);
        }
    }
}
Collection.main(null);


{"key":"Acura RL 2006","data":{"type":"Car","make":"Acura","model":"RL","year":2006,"color":"Silver}}
{"key":"Cadillac Boughman 1969","data":{"type":"Car","make":"Cadillac","model":"Boughman","year":1969,"color":"Black}}
{"key":"Cat Black 10","data":{"type":"Pet","name":"Cat","age":10,"color":"Black}}
{"key":"Dog Brown 14","data":{"type":"Pet","name":"Dog","age":14,"color":"Brown}}
{"key":"Ford Excursion 2003","data":{"type":"Car","make":"Ford","model":"Excursion","year":2003,"color":"Green}}
{"key":"Ford F-350 1997","data":{"type":"Car","make":"Ford","model":"F-350","year":1997,"color":"Green}}
{"key":"Ford Fusion 2015","data":{"type":"Car","make":"Ford","model":"Fusion","year":2015,"color":"Guard}}
{"key":"Kitty Calico 1","data":{"type":"Pet","name":"Kitty","age":1,"color":"Calico}}
{"key":"Lion Gold 8","data":{"type":"Pet","name":"Lion","age":8,"color":"Gold}}
{"key":"Pig Pink 3","data":{"type":"Pet","name":"Pig","age":3,"color":"Pink}}
{"key":"Robin Red 7","data":{"type":"Pet","nam

## Hacks
Build your own Collectable objects and sort them.